## Scenario 3: Tracking server on localhost with SQLite backend and remote (s3) artifacts store

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root s3://<YOUR_BUCKET>`

The exampe uses an s3 bucket to store the artifacts, so you'll to provide the bucket. Check the [mlflow documentation](https://mlflow.org/docs/latest/tracking.html#artifact-stores) for more options to store your artifacs.

In [ ]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
mlflow.list_experiments()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")

In [ ]:
mlflow.list_experiments()

### Interacting with the model registry

In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [ ]:
client.list_registered_models()

In [ ]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)